In [ ]:
# import required packages
import os
import pandas pd

In [ ]:
# References: 

- Mukhiya, S. K., & Ahmed, U. (2020). Hands-On Exploratory Data Analysis with Python: Perform EDA techniques to understand, summarize, and investigate your data. Packt Publishing Ltd.